In [1]:
import pyspark
from pyspark.sql import SparkSession



In [2]:
spark = SparkSession.builder \
    .master("spark://me-zoomcamp.us-central1-a.c.supple-comfort-444210-e6.internal:7077") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/18 19:50:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark


In [4]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [5]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [7]:

df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [8]:
common_colums = [
    'VendorID',
    'pickup_datetime',
    'dropoff_datetime',
    'store_and_fwd_flag',
    'RatecodeID',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'fare_amount',
    'extra',
    'mta_tax',
    'tip_amount',
    'tolls_amount',
    'improvement_surcharge',
    'total_amount',
    'payment_type',
    'congestion_surcharge'
]

In [10]:
from pyspark.sql import functions as F

In [11]:
df_green_sel = df_green \
    .select(common_colums) \
    .withColumn('service_type', F.lit('green'))

df_yellow_sel = df_yellow \
    .select(common_colums) \
    .withColumn('service_type', F.lit('yellow'))


df_trips_data = df_green_sel.unionAll(df_yellow_sel)


In [13]:
df_trips_data.show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|service_type|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|       2|2020-01-11 04:05:54|2020-01-11 04:13:49|                 N|       1.0|         129|         129|            1.0|         0.81|        6.5|  0.5|    0.5|      0.71|         0.0|       

In [14]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



In [15]:
df_result = (
    df_trips_data
    .withColumn("revenue_month", F.date_trunc('month', F.col('pickup_datetime')))
    .groupBy(
        "PULocationID",
        "revenue_month",
        "service_type"
    )
    .agg(
        F.sum('fare_amount').alias('revenue_monthly_fare'),
        F.sum('extra').alias('revenue_monthly_extra'),
        F.sum('mta_tax').alias('revenue_monthly_mta_tax'),
        F.sum('tip_amount').alias('revenue_monthly_tip_amount'),
        F.sum('tolls_amount').alias('revenue_monthly_tolls_amount'),
        F.sum('improvement_surcharge').alias('revenue_monthly_improvement_surcharge'),
        F.sum('total_amount').alias('revenue_monthly_total_amount'),
        F.sum('congestion_surcharge').alias('revenue_monthly_congestion_surcharge'),
        F.avg('passenger_count').alias('avg_montly_passenger_count'),
        F.avg('trip_distance').alias('avg_montly_trip_distance')
    )
)